<a href="https://colab.research.google.com/github/danriv16/danriv16/blob/main/Deterministic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 45.4 MB/s eta 0:00:00


In [ ]:
# save as humanitarian_gurobi.py and run: python humanitarian_gurobi.py
from gurobipy import Model, GRB, LinExpr, QuadExpr
import itertools

# ---------- Data (from your GAMS) ----------
i_list = ['i1','i2','i3']
j_list = ['j1','j2','j3','j4','j5']
t_list = ['t1','t2','t3']
m_list = ['m1','m2']

# Demand Dmj(m,j)  (note: GAMS table didn't vary over t)
Dmj = {
  ('m1','j1'): 215, ('m1','j2'):192, ('m1','j3'):331, ('m1','j4'):169, ('m1','j5'):318,
  ('m2','j1'): 235, ('m2','j2'):210, ('m2','j3'):364, ('m2','j4'):186, ('m2','j5'):349,
}

ICm = {'m1': 3.33, 'm2': 2.50}

Tdij = {
  ('i1','j1'):64, ('i1','j2'):64, ('i1','j3'):61, ('i1','j4'):67, ('i1','j5'):61,
  ('i2','j1'):30, ('i2','j2'):30, ('i2','j3'):30, ('i2','j4'):36, ('i2','j5'):30,
  ('i3','j1'):33, ('i3','j2'):33, ('i3','j3'):30, ('i3','j4'):36, ('i3','j5'):33,
}

Kmi = {
  ('m1','i1'):460, ('m1','i2'):385, ('m1','i3'):385,
  ('m2','i1'):505, ('m2','i2'):420, ('m2','i3'):420,
}

fi = {'i1':7580, 'i2':3790, 'i3':3790}

cm = {'m1':333, 'm2':250}

# small constant in denominator as in your GAMS
eps = 0.001

# ---------- Model ----------
model = Model('humanitarian_stage1')

# allow nonconvex quadratic terms (required for bilinear pi*Dj and rho*Dj_prev)
model.setParam('NonConvex', 2)

# Variables
X = {}      # integer distributed commodity item m from i to j at time t
for m,i,j,t in itertools.product(m_list, i_list, j_list, t_list):
    X[(m,i,j,t)] = model.addVar(vtype=GRB.INTEGER, name=f"X_{m}_{i}_{j}_{t}", lb=0)

y = {}
for i in i_list:
    y[i] = model.addVar(vtype=GRB.BINARY, name=f"y_{i}")

Dj = {}   # remaining demand Djtmz(m,j,t)
for m,j,t in itertools.product(m_list, j_list, t_list):
    Dj[(m,j,t)] = model.addVar(vtype=GRB.CONTINUOUS, name=f"Dj_{m}_{j}_{t}", lb=0)

rho = {}
for m,j,t in itertools.product(m_list, j_list, t_list):
    rho[(m,j,t)] = model.addVar(vtype=GRB.CONTINUOUS, name=f"rho_{m}_{j}_{t}", lb=0, ub=1)

pi = {}
for m,j,t in itertools.product(m_list, j_list, t_list):
    # pi can be continuous (no explicit lb in GAMS). Put lb=-GRB.INFINITY if you want unconstrained below 0.
    pi[(m,j,t)] = model.addVar(vtype=GRB.CONTINUOUS, name=f"pi_{m}_{j}_{t}")

# You can keep z as objective expression or var for clarity:
z = model.addVar(vtype=GRB.CONTINUOUS, name='z')

model.update()

# ---------- Constraints ----------

# 1) Capacity: sum_j X(m,i,*,t) = Kmi(m,i) * y_i   (for each m,i,t)
for m,i,t in itertools.product(m_list, i_list, t_list):
    lhs = LinExpr()
    for j in j_list:
        lhs.add(X[(m,i,j,t)])
    model.addConstr(lhs == Kmi[(m,i)], name=f"capacity_{m}_{i}_{t}")

# 2) Demand recursion:
# t=1: Dj(m,j,t1) = Dmj(m,j) - sum_i X(m,i,j,t1)
# t>1: Dj(m,j,t) = Dj(m,j,t-1) - sum_i X(m,i,j,t)
for m,j,t in itertools.product(m_list, j_list, t_list):
    lhs = Dj[(m,j,t)]
    rhs = LinExpr()
    if t == 't1':
        # constant Dmj
        rhs.addConstant(Dmj[(m,j)])
    else:
        # previous period var
        # find prev index:
        prev_index = 't' + str(int(t[1]) - 1)
        rhs.add(Dj[(m,j,prev_index)])
    # subtract sum of shipments in current period:
    for i in i_list:
        rhs.addTerm(-1.0, X[(m,i,j,t)])
    model.addConstr(lhs == rhs, name=f"demand_rec_{m}_{j}_{t}")

# 3) Dj >= 0 already enforced by var lb

# 4) Unsatisfied demand constraints:
# For t=1: rho >= 1 - sum_i X / Dmj  -> equivalently sum_i X + rho * Dmj <= Dmj (linear)
# For t>1: rho >= 1 - sum_i X / (Dj_prev + eps) -> sum_i X + rho*(Dj_prev+eps) <= (Dj_prev+eps)  (quadratic)
for m,j,t in itertools.product(m_list, j_list, t_list):
    # build s = sum_i X(m,i,j,t)
    s = LinExpr()
    for i in i_list:
        s.add(X[(m,i,j,t)])
    if t == 't1':
        denom = Dmj[(m,j)]
        # s + rho * denom <= denom -> linear (rho*denom is constant*var)
        expr = LinExpr(s)
        expr.add(rho[(m,j,t)], denom)     # adds denom * rho
        model.addConstr(expr <= denom, name=f"unsat_t1_{m}_{j}_{t}")
    else:
        prev_index = 't' + str(int(t[1]) - 1)
        denom_var = Dj[(m,j,prev_index)]
        # constraint: s + rho*(denom_var + eps) <= denom_var + eps
        # rearrange: s + rho*denom_var + rho*eps <= denom_var + eps
        # This includes bilinear term rho*denom_var -> quadratic constraint
        # We'll use addQConstr to add quadratic term coefficient
        # Build linear part: s + rho*eps - denom_var - eps <= 0  and quadratic part + rho*denom_var <= 0
        linpart = LinExpr()
        linpart += s
        linpart.add(rho[(m,j,t)], eps)   # rho*eps
        linpart.add(-1.0, denom_var)
        linpart.addConstant(-eps)
        # Now construct quadratic expression: rho * denom_var (coefficient +1)
        # addQConstr(quadexpr + linpart <= 0)
        qexpr = QuadExpr()
        qexpr.add(rho[(m,j,t)], denom_var, 1.0)  # adds 1 * rho * denom_var
        # combine: qexpr + linpart <= 0  -> qexpr + linpart <= 0
        model.addQConstr(qexpr + linpart <= 0, name=f"unsat_quadratic_{m}_{j}_{t}")

# 5) Pi (deprivation) estimation: 3 linear lower bounds
# pi >= 7 + 188.4*rho
# pi >= -125.3 + 453*rho
# pi >= -420.8 + 847*rho
for m,j,t in itertools.product(m_list, j_list, t_list):
    model.addConstr(pi[(m,j,t)] >= 7 + 188.4 * rho[(m,j,t)], name=f"pi1_{m}_{j}_{t}")
    model.addConstr(pi[(m,j,t)] >= -125.3 + 453.0 * rho[(m,j,t)], name=f"pi2_{m}_{j}_{t}")
    model.addConstr(pi[(m,j,t)] >= -420.8 + 847.0 * rho[(m,j,t)], name=f"pi3_{m}_{j}_{t}")

# 6) rho bounds 0..1 already enforced

# 7) Capacity equality was defined as sum_j X = Kmi* y. We already set equality to Kmi; now enforce yi linking:
# Wait: earlier capacity constraint set sum_j X == Kmi(m,i) * yi(i) — but in code I used sum_j X == Kmi(m,i) (forgot yi). Let's fix:
# Rebuild capacity constraints to include y. We will remove previous capacity constraints and re-add them correctly.

# Remove all capacity constraints added earlier to replace them
# (Easier approach: we named them 'capacity_...' earlier -> remove them)
for c in list(model.getConstrs()):
    if c.ConstrName and c.ConstrName.startswith("capacity_"):
        model.remove(c)
model.update()

for m,i,t in itertools.product(m_list, i_list, t_list):
    lhs = LinExpr()
    for j in j_list:
        lhs.add(X[(m,i,j,t)])
    # right-hand side Kmi(m,i) * y[i]
    rhs = LinExpr()
    rhs.add(y[i], Kmi[(m,i)])
    model.addConstr(lhs == rhs, name=f"capacity_{m}_{i}_{t}")

# 8) The Dj recursion was added earlier. Good.

# ---------- Objective ----------
# totcost.. z = sum((m,i,j,t), X* (cm(m) + Tdij(i,j))) + sum(i, fi(i)*y(i))
#           + sum((m,i,t), (Kmi(m,i) - sum_j X(m,i,j,t)) * ICm(m))
#           + 1*sum((m,j,t), pi(m,j,t) * Dj(m,j,t));

# Build linear part and quadratic part (pi * Dj)
lin = LinExpr()

# sum X*(cm + Tdij)
for m,i,j,t in itertools.product(m_list, i_list, j_list, t_list):
    coeff = cm[m] + Tdij[(i,j)]
    lin.add(X[(m,i,j,t)], coeff)

# sum fi*y
for i in i_list:
    lin.add(y[i], fi[i])

# sum (Kmi - sum_j X) * ICm  = sum Kmi*ICm  - sum_j X * ICm
# constant part:
const_term = 0.0
for m,i in itertools.product(m_list, i_list):
    const_term += Kmi[(m,i)] * ICm[m]
# subtract shipments * ICm
for m,i,j,t in itertools.product(m_list, i_list, j_list, t_list):
    lin.add(X[(m,i,j,t)], -ICm[m])

# set z equality by adding constraint z == (lin + const + sum(pi*Dj))
# But because pi*Dj is quadratic, build quadratic expression separate and then set z == lin + const + quad
quad = QuadExpr()
for m,j,t in itertools.product(m_list, j_list, t_list):
    quad.add(pi[(m,j,t)], Dj[(m,j,t)], 1.0)   # adds pi * Dj

# Create expression for RHS: linear + constant + quad
# We'll implement equality constraint z == linear + const + quad by moving all to LHS: z - linear - const - quad == 0
# Gurobi handles a quadratic expression in addQConstr of the form (QuadExpr + LinExpr) ...
rhs_lin = LinExpr(lin)
rhs_lin.addConstant(const_term)

# Build final quadratic + linear expression: (quad + rhs_lin) - z == 0  -> quad + rhs_lin - z == 0
# So addQConstr(quad + rhs_lin - z == 0)
final_lin = LinExpr(rhs_lin)
final_lin.addTerm(-1.0, z)

model.addQConstr(quad + final_lin == 0, name='totcost_eq')

# Also we can set objective simply to minimize z
model.setObjective(z, GRB.MINIMIZE)

# ---------- Model options ----------
model.setParam('TimeLimit', 300)  # example time limit (seconds); change as desired
model.setParam('MIPGap', 1e-4)

# ---------- Optimize ----------
model.optimize()

# ---------- Output ----------
if model.Status == GRB.OPTIMAL or model.Status == GRB.INTERRUPTED or model.Status == GRB.FEASIBLE:
    print("\nObjective z:", z.X)
    # print yi
    print("\nOpen facilities y:")
    for i in i_list:
        print(i, "=", y[i].X)
    # print some shipments
    print("\nSome X values (non-zero):")
    for (m,i,j,t), var in X.items():
        if var.X > 1e-6:
            print(f"X[{m},{i},{j},{t}] = {var.X}")
    # print Dj and pi and rho
    print("\nRemaining demands (Dj):")
    for (m,j,t), var in Dj.items():
        print(f"Dj[{m},{j},{t}] = {var.X:.3f}")
    print("\nrho and pi (examples):")
    for (m,j,t) in list(itertools.islice(list(itertools.product(m_list,j_list,t_list)), 12)):
        print(f"rho[{m},{j},{t}]={rho[(m,j,t)].X:.4f}, pi={pi[(m,j,t)].X:.4f}")
else:
    print("No feasible/optimal solution found. Model status:", model.Status)
